In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [2]:
temp = cudf.read_parquet('./cow_step_1_final.parquet/*')

In [3]:
temp

,GISJOIN,p_id,age,gender_0,gender_1,JRHE004,JRHE005,JRHE006,JRHE007,JRHE008,...,JRHE010,JRHE011,JRHE014,JRHE015,JRHE016,JRHE017,JRHE018,JRHE019,JRHE020,JRHE021
0,4001509505001,1372,1372,754,618,365,0,0,86,0,...,303,0,356,63,48,70,81,0,0,0
1,4001509506001,2688,2688,1276,1412,932,45,53,58,106,...,37,0,1363,0,0,0,0,0,49,0
2,4001509506002,982,982,422,560,353,0,0,38,0,...,0,0,431,0,34,37,58,0,0,0
3,4001509506003,1576,1576,711,865,566,0,0,0,0,...,145,0,391,0,273,160,0,41,0,0
4,4001509506004,624,624,345,279,84,0,181,43,37,...,0,0,203,0,0,47,0,0,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,253,66,57,23,43,...,78,0,211,42,39,142,7,64,48,0
216860,55000709604001,1228,1228,612,616,369,44,33,42,8,...,100,8,360,32,58,94,10,7,54,1
216861,55000709604002,582,582,331,251,189,16,0,64,9,...,53,0,157,8,30,13,5,0,38,0
216862,55000709604003,722,722,363,359,175,31,19,21,24,...,87,0,252,0,35,25,13,20,8,6


In [4]:
temp.index = temp.GISJOIN
temp.drop_column('GISJOIN')

Male - 0 <br>
Female - 1

## block wise - Male & Female -> cow
<b>JRHE001:</b>    Total</br>
<b>JRHE002:</b>    Male</br>
<b>JRHE004:</b>    Male: Private for-profit wage and salary workers: Employee of private company workers</br>
<b>JRHE005:</b>    Male: Private for-profit wage and salary workers: Self-employed in own incorporated business workers</br>
<b>JRHE006:</b>    Male: Private not-for-profit wage and salary workers</br>
<b>JRHE007:</b>    Male: Local government workers</br>
<b>JRHE008:</b>    Male: State government workers</br>
<b>JRHE009:</b>    Male: Federal government workers</br>
<b>JRHE010:</b>    Male: Self-employed in own not incorporated business workers</br>
<b>JRHE011:</b>    Male: Unpaid family workers</br>
<b>JRHE012:</b>    Female</br>
<b>JRHE014:</b>    Female: Private for-profit wage and salary workers: Employee of private company workers</br>
<b>JRHE015:</b>    Female: Private for-profit wage and salary workers: Self-employed in own incorporated business workers</br>
<b>JRHE016:</b>    Female: Private not-for-profit wage and salary workers</br>
<b>JRHE017:</b>    Female: Local government workers</br>
<b>JRHE018:</b>    Female: State government workers</br>
<b>JRHE019:</b>    Female: Federal government workers</br>
<b>JRHE020:</b>    Female: Self-employed in own not incorporated business workers</br>
<b>JRHE021:</b>    Female: Unpaid family workers</br>

# cow Mappings Final

<b>0</b>: Private for-profit wage and salary workers: Employee of private company workers</br>
<b>1</b>: Private for-profit wage and salary workers: Self-employed in own incorporated business workers</br>
<b>2</b>: Private not-for-profit wage and salary workers</br>
<b>3</b>: Local government workers</br>
<b>4</b>: State government workers</br>
<b>5</b>: Federal government workers</br>
<b>6</b>: Self-employed in own not incorporated business workers</br>
<b>7</b>: Unpaid family workers</br> 
<b>8</b>: Data not available/under 16 years

In [5]:
cow_by_age_cols_male = {
"JRHE004": 0,
"JRHE005": 1,
"JRHE006": 2,
"JRHE007": 3,
"JRHE008": 4,
"JRHE009": 5,
"JRHE010": 6,
"JRHE011": 7
}
cow_by_age_cols_female = {
"JRHE014": 0,
"JRHE015": 1,
"JRHE016": 2,
"JRHE017": 3,
"JRHE018": 4,
"JRHE019": 5,
"JRHE020": 6,
"JRHE021": 7
}

### Males

In [6]:
df = cudf.DataFrame()
for key in cow_by_age_cols_male:
    
    temp_df = cudf.DataFrame({
        'GISJOIN': temp.index.repeat(temp[key]).as_column()
    })
    temp_df['cow'] = cow_by_age_cols_male[key]
    df = cudf.concat([df,temp_df])

### Females

In [7]:
for key in cow_by_age_cols_female:
    
    temp_df = cudf.DataFrame({
        'GISJOIN': temp.index.repeat(temp[key]).as_column()
    })
    temp_df['cow'] = cow_by_age_cols_female[key]
    df = cudf.concat([df,temp_df])

In [8]:
df

,GISJOIN,cow
0,4001509505001,0
1,4001509505001,0
2,4001509505001,0
3,4001509505001,0
4,4001509505001,0
...,...,...
294227,55000709604004,7
294228,55000709604004,7
294229,55000709604004,7
294230,55000709604004,7


In [9]:
df.groupby(by='cow').count()

,GISJOIN
cow,
0,161573949
1,8509315
2,19202718
3,19432921
4,11522611
5,6529395
6,15896895
7,560104


In [10]:
df.to_parquet('cow_step_2.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "
